In [46]:
import numpy as np
import stanza

In [3]:
nlp_uk = stanza.Pipeline(lang="uk")

2020-06-09 15:42:06 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-06-09 15:42:06 INFO: Use device: gpu
2020-06-09 15:42:06 INFO: Loading: tokenize
2020-06-09 15:42:09 INFO: Loading: mwt
2020-06-09 15:42:09 INFO: Loading: pos
2020-06-09 15:42:10 INFO: Loading: lemma
2020-06-09 15:42:10 INFO: Loading: depparse
2020-06-09 15:42:11 INFO: Done loading processors!


In [4]:
import pandas as pd
import json

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
def save_data_to_json(data, file):
    with open(file, 'w') as f:
        json.dump(data, f)

In [7]:
def load_json(file):
    with open(file, 'r') as f:
        data = json.load(f)
        return data


In [8]:
def sort_dict_by_values(d, reverse=False):
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=reverse)}

In [9]:
def read_all_files_in_one_df(files):
    all_df = []
    for file in files:
        df = pd.read_csv(file)
        all_df.append(df)
    return pd.concat(all_df)    

### Load data

In [10]:
booking_files = [
    '/home/dbabenko/Data-Science/NLP/Detect-emotion-sentimental/dataset/booking/booking-train.csv',
    '/home/dbabenko/Data-Science/NLP/Detect-emotion-sentimental/dataset/booking/booking-test.csv',
    '/home/dbabenko/Data-Science/NLP/Detect-emotion-sentimental/dataset/booking/booking-val.csv',
]

In [11]:
booking_df = read_all_files_in_one_df(booking_files)

In [12]:
booking_df.head()

,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
0,Лише дівчата на рецепції - три рази мені мінял...,Лише дівчата на рецепції - три рази мені мінял...,"Все. Одного досвіду вистарчило, щоб більше сюд...",4.6,10.0,verhovina.uk.html,2
1,"Оформлення кімнати хороше, досить приємне, на ...","Оформлення кімнати хороше, досить приємне, на ...",Nan,9.2,10.0,verhovina.uk.html,5
2,"Усе відмінно, завдяки якісному сервісу ми завж...","Усе відмінно, завдяки якісному сервісу ми завж...",Рекомендую людям котрі подорожують власним тра...,10.0,10.0,verhovina.uk.html,5
3,Ціна/якість в принципі,Ціна/якість в принципі,"душова, трохи старий ремонт, але за 500 грн за...",7.5,10.0,verhovina.uk.html,4
4,"Приїхали з сином біля сьомої ранку, сонні та в...","Приїхали з сином біля сьомої ранку, сонні та в...",Nan,8.3,10.0,verhovina.uk.html,4


In [13]:
!ls dataset/google-booking-association/

google-booking-association.json  google-booking-hotels-train.json
google-booking-hotels-test.json


In [14]:
google_booking_dict_test = load_json('dataset/google-booking-association/google-booking-hotels-test.json')

In [15]:
booking_hotels_test = list(google_booking_dict_test.values())

In [16]:
booking_test_df = booking_df.loc[booking_df['hotel'].isin(booking_hotels_test)]

In [17]:
booking_test_df.head()

,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
1786,"Хороше розташування , приємна ціна.","Хороше розташування , приємна ціна.","В душі багато павуків, грибок, обідрані дивани...",5.8,10.0,kiev-hotel-rock-wall.uk.html,3
1787,в цілому добре.,"Готель хороший, в хорошому місці,гарні номери,...",але ліжко двоспальне не передбачає зсунуті два...,7.9,10.0,kiev-hotel-rock-wall.uk.html,4
1788,Гарний будинок.,Гарний будинок. Вид з вікна зачаровує. Затишно...,В душі грибок...Ручка в дверях Коломана.,9.2,10.0,kiev-hotel-rock-wall.uk.html,5
1789,Персонал взагалі не відповідає ніяким критеріям.,Nan,Персонал взагалі не відповідає ніяким критерія...,4.5,10.0,kiev-hotel-rock-wall.uk.html,2
1790,Все\n,Все,Нічого,10.0,10.0,kiev-hotel-rock-wall.uk.html,5


In [18]:
len(booking_test_df)

7894

In [19]:
booking_test_df.to_csv('dataset/booking/booking-test.csv')

In [20]:
booking_train_df = booking_df.loc[~booking_df['hotel'].isin(booking_hotels_test)]

In [21]:
len(booking_train_df)

126189

In [22]:
booking_train_df.to_csv('dataset/booking/booking-train.csv')

##  Calculate positve and negative dictionary 

In [27]:
text_ex = booking_train_df['pos_text'].values[0]

In [28]:
doc = nlp_uk(text_ex)
doc.sentences[0]

[
  {
    "id": "1",
    "text": "Лише",
    "lemma": "лише",
    "upos": "PART",
    "xpos": "Q",
    "head": 2,
    "deprel": "discourse",
    "misc": "start_char=0|end_char=4"
  },
  {
    "id": "2",
    "text": "дівчата",
    "lemma": "дівчина",
    "upos": "NOUN",
    "xpos": "Ncfpny",
    "feats": "Animacy=Anim|Case=Nom|Gender=Fem|Number=Plur",
    "head": 9,
    "deprel": "nsubj",
    "misc": "start_char=5|end_char=12"
  },
  {
    "id": "3",
    "text": "на",
    "lemma": "на",
    "upos": "ADP",
    "xpos": "Spsl",
    "feats": "Case=Loc",
    "head": 4,
    "deprel": "case",
    "misc": "start_char=13|end_char=15"
  },
  {
    "id": "4",
    "text": "рецепції",
    "lemma": "рецепція",
    "upos": "NOUN",
    "xpos": "Ncfsln",
    "feats": "Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing",
    "head": 2,
    "deprel": "nmod",
    "misc": "start_char=16|end_char=24"
  },
  {
    "id": "5",
    "text": "-",
    "lemma": "-",
    "upos": "PUNCT",
    "xpos": "U",
    "feats": "Punc

In [30]:
def acquire_sentiment_pos_for_text(nlp, text, result, supporeted_pos = ['ADJ', 'ADV']):
    doc = nlp(text)
    num_words = 0
    for sent in doc.sentences:
        for token in sent.to_dict():
            if 'upos' not in token:
                continue
            pos = token['upos']
            if pos == 'PUNCT':
                continue
                
            num_words += 1
            if pos in supporeted_pos:
                lemma = token['lemma']
                if pos not in result:
                    result[pos] = dict()
                    result[pos][lemma] = 1
                else:
                    if lemma not in result[pos]:
                        result[pos][lemma] = 1
                    else:
                        result[pos][lemma] += 1      
    return num_words

In [31]:
def acquire_sentiment_pos(nlp, texts):
    result = dict()
    total_num_words = 0
    for text in texts:
        if len(text) == 0:
            continue
        if text == 'Nan':
            continue
            
        total_num_words += acquire_sentiment_pos_for_text(nlp, text, result)
    
    for pos in result:
        result[pos] = {k: v for k, v in sorted(result[pos].items(), key=lambda item: item[1], reverse=True)}
    
    return result, total_num_words 

In [38]:
%time pos_dict, total_pos_num_words = acquire_sentiment_pos(nlp_uk, booking_train_df['pos_text'].values)

CPU times: user 1h 54min 43s, sys: 14.5 s, total: 1h 54min 58s
Wall time: 1h 55min 1s


In [39]:
total_pos_num_words

2012137

In [40]:
save_data_to_json(pos_dict, 'dataset/booking/pos_words.json')

In [41]:
%time neg_dict, total_neg_num_words = acquire_sentiment_pos(nlp_uk, booking_train_df['neg_text'].values)

CPU times: user 1h 35min 51s, sys: 11.4 s, total: 1h 36min 3s
Wall time: 1h 35min 57s


In [42]:
total_neg_num_words

2159506

In [43]:
save_data_to_json(neg_dict, 'dataset/booking/neg_words.json')

## Calculate PMI for adjectives abd verbs in existing corpus

In [44]:
neg_dict

{'ADV': {'дуже': 20213,
  'так': 7525,
  'можна': 5821,
  'трохи': 4144,
  'ще': 3998,
  'коли': 3976,
  'тому': 3850,
  'потрібно': 3652,
  'як': 3617,
  'вже': 3435,
  'холодно': 2702,
  'там': 2687,
  'чутно': 2559,
  'взагалі': 2558,
  'більше': 2493,
  'вночі': 2402,
  'добре': 2355,
  'погано': 2271,
  'треба': 2239,
  'шумно': 2201,
  'де': 2043,
  'досить': 1967,
  'зовсім': 1957,
  'неможливо': 1866,
  'краще': 1818,
  'далеко': 1774,
  'теж': 1735,
  'також': 1731,
  'вранці': 1659,
  'мало': 1517,
  'поруч': 1517,
  'особливо': 1419,
  'постійно': 1395,
  'зручно': 1383,
  'можливо': 1291,
  'потім': 1206,
  'тут': 1196,
  'відразу': 1179,
  'сильно': 1154,
  'довго': 1078,
  'важко': 1078,
  'ввечері': 1052,
  'поки': 1016,
  'прохолодно': 1009,
  'чисто': 994,
  'багато': 992,
  'незручно': 979,
  'більш': 960,
  'варто': 932,
  'абсолютно': 905,
  'завжди': 901,
  'занадто': 899,
  'складно': 883,
  'майже': 874,
  'прямо': 866,
  'раніше': 840,
  'нормально': 837,
  'бру

In [53]:
len(set(pos_dict['ADJ'].keys()).union(set(neg_dict['ADJ'].keys())))

9266

In [65]:
def calc_sentiment_score(pos_dict, neg_dict, total_pos, total_neg, word):
    freq_pos_w = pos_dict[word] if word in pos_dict else 0.1
    freq_neg_w = neg_dict[word] if word in neg_dict else 0.1
    
    sentiment_score = np.log2((freq_pos_w * total_neg) / (freq_neg_w * total_pos))
    return sentiment_score

In [66]:
calc_sentiment_score(pos_dict['ADJ'], neg_dict['ADJ'], total_pos_num_words, total_neg_num_words, 'привітний')

4.70307015851435

#### Calc sentiment score for adjective

In [ ]:
all_adjectives = set(pos_dict['ADJ'].keys()).union(set(neg_dict['ADJ'].keys()))

In [73]:
adjective_scores = dict()
for adj in all_adjectives:
    score = calc_sentiment_score(pos_dict['ADJ'], neg_dict['ADJ'], total_pos_num_words, total_neg_num_words, adj)
    adjective_scores[adj] = score
    
adjective_scores = sort_dict_by_values(adjective_scores, reverse=True)

In [75]:
adjective_scores['брудний']

-4.466733822007981

In [76]:
adjective_scores['чистий']

3.8180864768644764

In [88]:
save_data_to_json(adjective_scores, 'data/adj-sent-scores.json')

#### Calc sentiment score for adverbs

In [77]:
all_adverbs = set(pos_dict['ADV'].keys()).union(set(neg_dict['ADV'].keys()))

In [81]:
adverbs_scores = dict()
for adv in all_adverbs:
    score = calc_sentiment_score(pos_dict['ADV'], neg_dict['ADV'], total_pos_num_words, total_neg_num_words, adv)
    adverbs_scores[adv] = score
    
adverbs_scores = sort_dict_by_values(adverbs_scores, reverse=True)

In [83]:
adverbs_scores['стильно']

5.083825441244996

In [87]:
adverbs_scores['дорого']

-0.11403130541864527

In [89]:
save_data_to_json(adverbs_scores, 'data/adv-sent-scores.json')